In [1]:
# Рисовать графики сразу же
%matplotlib inline

import pandas as pd
import numpy as np

import os

import matplotlib.pyplot as plt

from Get_model_times import times

plt.rc('xtick', labelsize=14) 
plt.rc('ytick', labelsize=14) 

plt.style.use('ggplot')  # Красивые графики
plt.rcParams['figure.figsize'] = (15, 5)  # Размер картинок
plt.rc('font', family = 'Arial')

### Рассчитываем суточный сток всех рек

***Загружаем .xlsx-файл со среднемесячными данными стока рек***

In [2]:
Cur_dir = os.getcwd()
Monthly_Flux = pd.read_excel(Cur_dir + r'\input_data\Dis_Var_1.xlsx',
                             index_col = 0)
Monthly_Flux.head()

,P1 - flux/discharge rate
date and time,
2025-01-01 12:00:00,0.030
2025-02-01 12:00:00,0.028
2025-03-01 12:00:00,0.027
2025-04-01 12:00:00,0.034
2025-05-01 12:00:00,0.038


***Конвертируем среднемесячные данные в суточные значения***

In [3]:
Daily_flux = Monthly_Flux.resample('D').sum()
Daily_flux1 = Daily_flux.reindex(pd.date_range(start='2025-01-01',
                                               end='2025-12-31',
                                               freq='1D'))
Daily_Flux2 = Daily_flux1.fillna(method = 'ffill')

In [4]:
Daily_Flux2.head()

,P1 - flux/discharge rate
2025-01-01,0.03
2025-01-02,0.03
2025-01-03,0.03
2025-01-04,0.03
2025-01-05,0.03


***Сохраняем суточный сток в .xlsx - файл***

In [5]:
Daily_Flux2.to_excel(Cur_dir + r'\input_data\Dis_Var_1_daily.xlsx')

***Возвращаем имена столбцов фрейма данных суточного стока***

In [5]:
Daily_Flux2.columns

Index(['P1 - flux/discharge rate'], dtype='object')

***Конвертируем столбец фрейма в одномерный numpy-массив***

In [6]:
Discharge = np.c_[Daily_Flux2.values]
Discharge[:5]

array([[0.03],
       [0.03],
       [0.03],
       [0.03],
       [0.03]])

### Рассчитываем соленость речной и морской воды

***Соленость речной воды для одного притока рассчитывается по следующей формуле:***




${\large S_{river}= 1.218 * Q_{river}^{-0.34}}$

In [7]:
Salin_river = 1.218 * (np.c_[Discharge] **(-0.34))
Salin_river[Salin_river == np.inf] = 0  # В случае отсутствия стока, принимаем значения солености равными 0
Salin_river[:5]

array([[4.01260178],
       [4.01260178],
       [4.01260178],
       [4.01260178],
       [4.01260178]])

***Соленость морской воды***

In [8]:
Table_salin_sea = pd.read_excel(Cur_dir + r'\input_data\Salin_sea_water.xlsx',
                                index_col = 0)
Table_salin_sea.head()

,T2 - Salinity
date and time,
2025-01-01 12:00:00,14.61
2025-02-01 12:00:00,14.42
2025-03-01 12:00:00,14.17
2025-04-01 12:00:00,15.25
2025-05-01 12:00:00,0.00


In [9]:
Salin_sea = Table_salin_sea.resample('D').sum()
Salin_sea1 = Salin_sea.reindex(pd.date_range(start='2025-01-01',
                                               end='2025-12-31',
                                               freq='1D'))
Salin_sea2 = Salin_sea1.fillna(method = 'ffill')
Salin_sea2.head()

,T2 - Salinity
2025-01-01,14.61
2025-01-02,14.61
2025-01-03,14.61
2025-01-04,14.61
2025-01-05,14.61


### Рассчитываем температуру речной воды (для всего года)

***Загружаем данные о температуре воздуха во фрейм***

In [10]:
Tabletemp = pd.read_excel(Cur_dir + r'\input_data\Копия_Метео_сут_период1990-2030_NEW.xls',
                          sheet_name = '2025',
                          header = 1,
                          index_col = 0,
                          use_cols = "A, B",
                          convert_float = True)
Tabletemp.head()

,"Tемп-ра, ºС"
2025-01-01,4.708917
2025-01-02,1.028772
2025-01-03,-2.855933
2025-01-04,-0.769141
2025-01-05,0.501886


***Рассчитываем температуру воды согласно следующей формуле:***

${\large T_{river} = 0.62 * T_{air} +4.4}$

In [11]:
Temp = Tabletemp.values   #Конвертируем в numpy-массив
Temp_river = 0.62 * Temp + 4.4
Temp_river[Temp_river < 0] = 0 #Заменяем отрицательные значения температуры на 0
Temp_river[:5]

array([[7.31952869],
       [5.03783862],
       [2.62932178],
       [3.92313281],
       [4.71116931]])

### Генерируем входящий файл с данными о расходах для модели

***Вызываем функцию формирования массива дат для входящего файла***

In [12]:
timedata = times(720, 1440, 365)
timedata.size

365

***Сохраняем все данные в ASCII-файл***

In [13]:
f_handle = open(Cur_dir + r'\input_data\dis_data.dis', 'ab')

np.savetxt(f_handle, np.c_[timedata, Discharge, Salin_river, Temp_river],
           fmt='%.7e', delimiter=' ', newline='\n',
           header= '\n'.join(["table-name           'Discharge : 1'", 
                              "contents             'regular   '", 
                              "location             'P1             '", 
                              "time-function        'non-equidistant'", 
                              "reference-time       20250101", 
                              "time-unit            'minutes'", 
                              "interpolation        'linear'", 
                              "parameter            'time                '                     unit '[min]'", 
                              "parameter            'flux/discharge rate '                     unit '[m3/s]'", 
                              "parameter            'Salinity            '                     unit '[ppt]'", 
                              "parameter            'Temperature         '                     unit '[°C]'", 
                              "records-in-table     365"]),
           comments='')
f_handle.close()